Import

In [ ]:
# import pandas as pd

# # Load the CSV file
# input_file = 's_aes_Quadruplets.csv'  # Replace with your input file name
# df = pd.read_csv(input_file)

# # Select the first 1000 records
# df_first_10000 = df.head(10000)

# # Save the selected records to another CSV file
# output_file = 'output_first_10000.csv'  # Replace with your desired output file name
# df_first_10000.to_csv(output_file, index=False)

# print(f"First 1000 records saved to {output_file}")


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SimpleRNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from keras.layers import SimpleRNN, Dense

Upload the data

In [ ]:
data = pd.read_csv('output_first_250000.csv')


Data prepration

In [ ]:

# Convert all hexadecimal entries to binary representation in the entire dataframe

y = data[['key']]

x = data.drop(['key','before_xor'], axis=1)
def convert_to_binary(hex_number):
    return bin(int(hex_number, 16))[2:].zfill(16)

x = x.applymap(convert_to_binary)
y = y.applymap(convert_to_binary)





X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

plaintext_train = np.array([list(map(int, list(row))) for row in X_train['plaintext'].values])
ciphertext_train = np.array([list(map(int, list(row))) for row in X_train['ciphertext'].values])
iv_train = np.array([list(map(int, list(row))) for row in X_train['iv'].values])

plaintext_test = np.array([list(map(int, list(row))) for row in X_test['plaintext'].values])
ciphertext_test = np.array([list(map(int, list(row))) for row in X_test['ciphertext'].values])
iv_test = np.array([list(map(int, list(row))) for row in X_test['iv'].values])

# Stack the arrays to create the transformed datasets
X_train_transformed = np.stack([plaintext_train, ciphertext_train, iv_train], axis=-1)
X_test_transformed = np.stack([plaintext_test, ciphertext_test, iv_test], axis=-1)

# For the y_train and y_test
key_train = np.array([list(map(int, list(row))) for row in y_train['key'].values])
key_test = np.array([list(map(int, list(row))) for row in y_test['key'].values])

y_train_transformed = key_train[..., np.newaxis]
y_test_transformed = key_test[..., np.newaxis]






<ipython-input-82-9695ef7f73ae>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  x = x.applymap(convert_to_binary)
<ipython-input-82-9695ef7f73ae>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  y = y.applymap(convert_to_binary)


In [ ]:
y_test.shape

(25000, 1)

Build the model

600
300
600
600
400

In [ ]:
# Build the SimpleRNN model

# model = Sequential()
# #mlp
# #model.add(Dense(256, activation='relu'))

# model.add(SimpleRNN(16, input_shape=(X_train_transformed.shape[1], X_train_transformed.shape[2])))
# #model.add(Dense(256, activation='relu'))

# model.add(Dense(16, activation='sigmoid'))

# # Compile and train the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(X_train_transformed, y_train_transformed, epochs=500, batch_size=y_test_transformed.shape[0], verbose=1)


#BidirectionalRNN
model = keras.Sequential()

# Add a Dense layer
model.add(layers.Dense(500, activation='relu'))

# Convert SimpleRNN to Bidirectional RNN
model.add(layers.Bidirectional(layers.SimpleRNN(500), input_shape=(X_train_transformed.shape[1], X_train_transformed.shape[2])))
#
# Add a Dense layer
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(500, activation='relu'))


# Add a Dense layer with Sigmoid activation
model.add(layers.Dense(16, activation='relu'))

In [ ]:
# import tensorflow as tf
# model = tf.keras.models.load_model('test.h5')
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(X_train_transformed, y_train_transformed, epochs=400, batch_size=y_test_transformed.shape[0], verbose=1)
model.summary()

Epoch 1/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0557 - loss: 0.4743
Epoch 2/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.0436 - loss: 0.2617
Epoch 3/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0753 - loss: 0.2526
Epoch 4/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0558 - loss: 0.2511
Epoch 5/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0526 - loss: 0.2506
Epoch 6/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0295 - loss: 0.2503
Epoch 7/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0368 - loss: 0.2502
Epoch 8/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0690 - loss: 0.2501
Epoch 9/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0747 - loss: 0.2501
Epoch 10/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0636 - loss: 0.2501
Epoch 11/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0572 - loss: 0.2501
Epoch 12/400
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.1404 - loss: 0.2501
E

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                     │ (25000, 16, 500)            │           2,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ (25000, 1000)               │       1,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (25000, 500)                │         500,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (25000, 500)                │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (25000, 16)                 │           8,016 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,286,050 (20.16 MB)

 Trainable params: 1,762,016 (6.72 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,524,034 (13.44 MB)

In [ ]:
print(len(model1.layers))

In [ ]:
model.save('test.h5')


In [1]:
import tensorflow as tf
model1 = tf.keras.models.load_model('test.h5')

OSError: Unable to synchronously open file (truncated file: eof = 10485760, sblock->base_addr = 0, stored_eof = 21203320)

Evaluate the model

In [ ]:

# Evaluate the model

_, accuracy = model.evaluate(X_test_transformed, y_test_transformed)
print(f"Model Evaluation Accuracy: {accuracy}")

# Make predictions
y_pred = (model.predict(X_test_transformed) > 0.5).astype("int32")

# # Calculate accuracy of predictions
# y_test_transformed=y_test_transformed.reshape(y_test_transformed.shape[0],16)
# accuracy = accuracy_score(np.ravel(y_test_transformed), np.ravel(y_pred))
# print(f"Accuracy Score: {accuracy}")
# # accuracy = accuracy_score(y_test_transformed, y_pred)
# # print(f"Accuracy Score: {accuracy}")



782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0633 - loss: 0.0483
Model Evaluation Accuracy: 0.06352656334638596
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step


In [ ]:
y_test_transformed.shape

In [ ]:
y_test_transformed=y_test_transformed.reshape(y_test_transformed.shape[0],16)
print(y_test_transformed.shape[0])

25000


In [ ]:
y_pred.shape

In [ ]:
y_test_transformed

In [ ]:
y_pred

In [ ]:
correct_count = 0

# Iterate over each array
for i in range(y_test_transformed.shape[0]):
    if np.array_equal(y_test_transformed[i], y_pred[i]):
        correct_count += 1

# Calculate accuracy
total_arrays = y_test_transformed.shape[0]
accuracy = correct_count / total_arrays

print(f"Number of correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Number of correct predictions: 18673
Accuracy: 74.69%


In [ ]:

_, accuracy = model.evaluate(X_train_transformed, y_train_transformed)
print(f"Model Evaluation Accuracy: {accuracy}")

# Make predictions
y_pred_ = (model.predict(X_train_transformed) > 0.5).astype("int32")

y_train_transformed=y_train_transformed.reshape(y_train_transformed.shape[0],16)
print(y_train_transformed.shape[0])
correct_count = 0

# Iterate over each array
for i in range(y_train_transformed.shape[0]):
    if np.array_equal(y_train_transformed[i], y_pred_[i]):
        correct_count += 1

# Calculate accuracy
total_arrays = y_train_transformed.shape[0]
accuracy = correct_count / total_arrays

print(f"Number of correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")

7032/7032 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.0624 - loss: 0.0235
Model Evaluation Accuracy: 0.06255528330802917
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step
225000
Number of correct predictions: 198670
Accuracy: 88.30%
